In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
print(os.getenv("LANGCHAIN_TRACING_V2"))
print(os.getenv("LANGCHAIN_API_KEY"))
def get_model():
    from langchain_community.chat_models.zhipuai import ChatZhipuAI
    from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage, AIMessage
    model = ChatZhipuAI(model='glm-4',
                        # temperature 需要精确到小数后两位，太多的话接口会报错
                        temperature=0.7,
                        api_key='00d2ffc1109bb54764d67a63c7c3826c.CWRsU68TB9pbmeig', max_tokens=1024)
    return model
model = get_model()
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

true
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


In [3]:
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage,AIMessage


messages = [
    SystemMessage(content="你是一个翻译官，帮我将下面的信息翻译为意大利语。并且只返回翻译的内容"),
    # AIMessage(content="好的"),
    HumanMessage(content="翻译：你好。我是一个学生"),
]

In [4]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
chain = model | parser
chain.invoke(messages)

'Il tuo messaggio in italiano:\n\nCiao. Sono uno studente.'

In [5]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "你是一个翻译官，帮我将下面的信息翻译为 {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
result = prompt_template.invoke({"language": "意大利语", "text": "你好。我是一个学生"})

result

ChatPromptValue(messages=[SystemMessage(content='你是一个翻译官，帮我将下面的信息翻译为 意大利语:'), HumanMessage(content='你好。我是一个学生')])

In [6]:
result.to_messages()

[SystemMessage(content='你是一个翻译官，帮我将下面的信息翻译为 意大利语:'),
 HumanMessage(content='你好。我是一个学生')]

In [7]:
chain = prompt_template | model | parser
chain.invoke({"language": "意大利语", "text": "我是一个学生"})

"Certainly! Please provide the information you'd like to have translated into Italian, and I'll do my best to assist you."

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke(
    [
        HumanMessage(content="你好， 我是bob"),
        AIMessage(content="你好"),
        HumanMessage(content="我叫什么"),
    ]
)
config = {"configurable": {"session_id": "abc2"}}
response = with_message_history.invoke(
    [HumanMessage(content="你好，我是bob")],
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

In [ ]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content